In [1]:
import numpy as np
import pandas as pd


def coord_convert(x,y,z):
    
    phi = np.arctan2(y, x)
    theta = np.arctan2(np.sqrt(x ** 2 + y ** 2), z)
    eta = -np.log(np.tan(theta/2))    
    return phi, eta, z


def sector_splitter(df, n, m):
    
    phi_bins = np.linspace(-np.pi, np.pi, n+1)  # bins for phi angles
    eta_bins = np.linspace(-4.5, 4.5, m+1)  # bins for eta angles
        
    df_list = []
    for i in range(n):
        for j in range(m):
            
            phi_mask = (phi_bins[i] < df['phi']) & (df['phi'] < phi_bins[i+1])
            eta_mask = (eta_bins[j] < df['eta']) & (df['eta'] < eta_bins[j+1])
            
            df_list.append(df[(phi_mask & eta_mask)])
                    
    return df_list


def data_puller(datamax):
    
    directory = '/home/aaportel/teams/group-3/data/'
    folders = ['train_1/','train_2/','train_5/']

    data_cutoff = 0
    data = []
   
    for folder in folders:
       
        hit_files = sorted([f for f in os.listdir(directory + folder) if f.endswith('hits.csv')])   
        truth_files = sorted([f for f in os.listdir(directory + folder) if f.endswith('truth.csv')])   
        
        for hit_file, truth_file in zip(hit_files, truth_files):

            # read a CSV file into a DataFrame
            X = pd.read_csv(directory + folder + hit_file, usecols=['x','y','z'])
            Y = pd.read_csv(directory + folder + truth_file, usecols=['particle_id'])
            
            # calculate phi, eta, and z from x, y, and z
            phi, eta, z = coord_convert(X['x'], X['y'],X['z'])

            # create a new DataFrame with phi, theta, eta, z and particle_id columns
            df = pd.DataFrame({
                'phi': phi,
                'eta': eta,
                'z': z,
                'particle_id': Y['particle_id'] 
            })

            # splits dataframe into chunks based on detector geometry
            n, m = 8, 4
            df_list = sector_splitter(df, n, m)
            
            # appends elements of df_list into a mega list
            data.extend(df_list)
            
            data_cutoff += 1
            if data_cutoff > datamax:
                return data
            
    return data

In [2]:
import torch
from torch_geometric.data import Data, Dataset
import os
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, dataframes, use_cuda = False):
        self.dataframes = dataframes
        self.use_cuda = use_cuda
        
    def __len__(self):
        return len(self.dataframes)
    
    def __getitem__(self, idx):

        df = self.dataframes[idx]
        
        input_data = df[['phi','eta','z']]
        output_data = df['particle_id']

        # Create a tensor of node features by stacking the columns of the input data
        node_features = torch.tensor(input_data.values).half()
        output_features = torch.tensor(output_data.values).half()

        phi = torch.from_numpy(input_data[['phi']].values)
        eta = torch.from_numpy(input_data[['eta']].values)

        if self.use_cuda:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            phi = phi.to(device)
            eta = eta.to(device)

        # Compute the pairwise differences between the phi and eta columns of adjacent nodes
        phi_diff = phi.unsqueeze(0) - phi.unsqueeze(-1)
        eta_diff = eta.unsqueeze(0) - eta.unsqueeze(-1)

        #this is deltaR
        diff_norm = torch.sqrt(phi_diff**2 + eta_diff**2)

        # Create a binary adjacency matrix based on a threshold of 1.7
        adjacency_matrix = torch.where(
            (diff_norm < 1.7) & (phi_diff > 0) & (eta_diff > 0), 
            torch.ones_like(diff_norm), 
            torch.zeros_like(diff_norm)
        )

        # Convert the adjacency matrix to a list of edge indices
        edge_indices = adjacency_matrix.squeeze(-1).nonzero(as_tuple=False).t()


        # Create a tensor of edge features by concatenating the phi and eta differences    
        phi_diff = phi_diff[edge_indices[0], edge_indices[1]]
        eta_diff = eta_diff[edge_indices[0], edge_indices[1]]
        edge_features = torch.cat((
            phi_diff.unsqueeze(-1),
            eta_diff.unsqueeze(-1)
        ), -1)

        if self.use_cuda:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            node_features = node_features.to(device) 
            edge_indices = edge_indices.to(device) 
            edge_features = edge_features.to(device)
            output_features = output_features.to(device)   
            
        # Create a PyTorch Geometric Data object
        data = Data(
            x          = node_features, 
            edge_index = edge_indices, 
            edge_attr  = edge_features,
            y          = output_features
        )
        return data

In [3]:
from torch import Tensor, nn
from torch import Tensor as T
from torch import Tensor

In [4]:
class MLP(nn.Module):
    def __init__(
        self,
        input_size,
        output_size,
        hidden_dim,
        L=3,
        *,
        bias=True,
        include_last_activation=False,
    ):
        """Multi Layer Perceptron, using ReLu as activation function.
        Args:
            input_size: Input feature dimension
            output_size:  Output feature dimension
            hidden_dim: Feature dimension of the hidden layers
            L: Total number of layers (1 initial layer, L-2 hidden layers, 1 output
                layer)
            bias: Include bias in linear layer?
            include_last_activation: Include activation function for the last layer?
        """
        super().__init__()
        layers = [nn.Linear(input_size, hidden_dim, bias=bias)]
        for _l in range(1, L - 1):
            layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_dim, hidden_dim, bias=bias))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_dim, output_size, bias=bias))
        if include_last_activation:
            layers.append(nn.ReLU())
        self.layers = nn.ModuleList(layers)

    def reset_parameters(self):
        for layer in self.layers:
            if hasattr(layer, "reset_parameters"):
                layer.reset_parameters()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [5]:
from torch.nn.functional import relu

@torch.jit.script
def _convex_combination(*, delta: T, residue: T, alpha_residue: float) -> T:
    return alpha_residue * residue + (1 - alpha_residue) * relu(delta)


_IDENTITY = nn.Identity()


def convex_combination(
    *,
    delta: T,
    residue: T,
    alpha_residue: float,
    residue_encoder: nn.Module = _IDENTITY,
) -> T:
    """Convex combination of ``relu(delta)`` and the residue."""
    if math.isclose(alpha_residue, 0.0):
        return relu(delta)
    assert 0 <= alpha_residue <= 1
    return _convex_combination(
        delta=delta, residue=residue_encoder(residue), alpha_residue=alpha_residue
    )


class ResIN(nn.Module):
    def __init__(
        self,
        layers: list[nn.Module],
        *,
        length_concatenated_edge_attrs: int,
        alpha: float = 0.5,
    ):
        """Apply a list of layers in sequence with residual connections for the nodes.
        Built for interaction networks, but any network that returns a node feature
        tensor and an edge feature tensor should
        work.
        Note that a ReLu activation function is applied to the node result of the
        layer.
        Args:
            layers: List of layers
            length_concatenated_edge_attrs: Length of the concatenated edge attributes
                (from all the different layers)
            alpha: Strength of the node embedding residual connection
        """
        super().__init__()
        self._layers = nn.ModuleList(layers)
        self._alpha = alpha
        #: Because of the residual connections, we need map the output of the previous
        #: layer to the dimension of the next layer (if they are different). This
        #: can be done with these encoders.
        self._residue_node_encoders = nn.ModuleList([nn.Identity() for _ in layers])
        self.length_concatenated_edge_attrs = length_concatenated_edge_attrs

    @staticmethod
    def _get_residue_encoder(
        *, in_dim: int, out_dim: int, hidden_dim: int
    ) -> nn.Module:
        if in_dim == out_dim:
            return nn.Identity()
        return MLP(
            input_size=in_dim,
            output_size=out_dim,
            hidden_dim=hidden_dim,
            include_last_activation=True,
            L=2,
        )

    @classmethod
    def identical_in_layers(
        cls,
        *,
        node_indim: int,
        edge_indim: int,
        node_hidden_dim: int,
        edge_hidden_dim: int,
        node_outdim=3,
        edge_outdim=4,
        object_hidden_dim=40,
        relational_hidden_dim=40,
        alpha: float = 0.5,
        n_layers=1,
    ):
        """Create a ResIN with identical layers of interaction networks except for
        the first and last one (different dimensions)
        If the input/hidden/output dimensions for the nodes are not the same, MLPs are
        used to map the previous output for the residual connection.
        Args:
            node_indim: Node feature dimension
            edge_indim: Edge feature dimension
            node_hidden_dim: Node feature dimension for the hidden layers
            edge_hidden_dim: Edge feature dimension for the hidden layers
            node_outdim: Output node feature dimension
            edge_outdim: Output edge feature dimension
            object_hidden_dim: Hidden dimension for the object model MLP
            relational_hidden_dim: Hidden dimension for the relational model MLP
            alpha: Strength of the residual connection
            n_layers: Total number of layers
        """
        node_dims = [
            node_indim,
            *[node_hidden_dim for _ in range(n_layers - 1)],
            node_outdim,
        ]
        edge_dims = [
            edge_indim,
            *[edge_hidden_dim for _ in range(n_layers - 1)],
            edge_outdim,
        ]
        assert len(node_dims) == len(edge_dims) == n_layers + 1
        layers = [
            InteractionNetwork(
                node_indim=node_dims[i],
                edge_indim=edge_dims[i],
                node_outdim=node_dims[i + 1],
                edge_outdim=edge_dims[i + 1],
                node_hidden_dim=object_hidden_dim,
                edge_hidden_dim=relational_hidden_dim,
            )
            for i in range(n_layers)
        ]
        length_concatenated_edge_attrs = edge_hidden_dim * (n_layers - 1) + edge_outdim
        mod = cls(
            layers,
            length_concatenated_edge_attrs=length_concatenated_edge_attrs,
            alpha=alpha,
        )
        mod._residue_node_encoders = nn.ModuleList(
            [
                cls._get_residue_encoder(
                    in_dim=node_dims[i],
                    out_dim=node_dims[i + 1],
                    hidden_dim=node_dims[i + 1],
                )
                for i in range(n_layers)
            ]
        )
        return mod

    def forward(
        self, x, edge_index, edge_attr
    ) -> tuple[Tensor, list[Tensor], list[Tensor]]:
        """Forward pass
        Args:
            x: Node features
            edge_index:
            edge_attr: Edge features
        Returns:
            node embedding, node embedding at each layer (including the input and
            final node embedding), edge embedding at each layer (including the input)
        """
        edge_attrs = [edge_attr]
        xs = [x]
        for layer, re in zip(self._layers, self._residue_node_encoders):
            delta_x, edge_attr = layer(x, edge_index, edge_attr)
            x = convex_combination(
                delta=delta_x, residue=x, alpha_residue=self._alpha, residue_encoder=re
            )
            xs.append(x)
            edge_attrs.append(edge_attr)
        return x, xs, edge_attrs

In [6]:
from torch_geometric.nn import MessagePassing

class InteractionNetwork(MessagePassing):
    def __init__(
        self,
#         *,
        node_indim: int,
        edge_indim: int,
        node_outdim=3,
        edge_outdim=4,
        node_hidden_dim=40,
        edge_hidden_dim=40,
        aggr="add",
    ):
        """Interaction Network, consisting of a relational model and an object model,
        both represented as MLPs.
        Args:
            node_indim: Node feature dimension
            edge_indim: Edge feature dimension
            node_outdim: Output node feature dimension
            edge_outdim: Output edge feature dimension
            node_hidden_dim: Hidden dimension for the object model MLP
            edge_hidden_dim: Hidden dimension for the relational model MLP
            aggr: How to aggregate the messages
        """
        super().__init__(aggr=aggr, flow="source_to_target")
        self.relational_model = MLP(
            2 * node_indim + edge_indim,
            edge_outdim,
            edge_hidden_dim,
        )
        self.object_model = MLP(
            node_indim + edge_outdim,
            node_outdim,
            node_hidden_dim,
        )
        self._e_tilde: T | None = None

    def forward(self, x: T, edge_index: T, edge_attr: T) -> tuple[T, T]:
        """Forward pass
        Args:
            x: Input node features
            edge_index:
            edge_attr: Input edge features
        Returns:
            Output node embedding, output edge embedding
        """
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        assert self._e_tilde is not None  # mypy
        return x_tilde, self._e_tilde

    # noinspection PyMethodOverriding
    def message(self, x_i: T, x_j: T, edge_attr: T) -> T:
        """Calculate message of an edge
        Args:
            x_i: Features of node 1 (node where the edge ends)
            x_j: Features of node 2 (node where the edge starts)
            edge_attr: Edge features
        Returns:
            Message
        """
        m = torch.cat([x_i, x_j, edge_attr], dim=1)
        self._e_tilde = self.relational_model(m)
        assert self._e_tilde is not None  # mypy
        return self._e_tilde

    # noinspection PyMethodOverriding
    def update(self, aggr_out: T, x: T) -> T:
        """Update for node embedding
        Args:
            aggr_out: Aggregated messages of all edges
            x: Node features for the node that receives all edges
        Returns:
            Updated node features/embedding
        """
        c = torch.cat([x, aggr_out], dim=1)
        return self.object_model(c)

In [7]:
class EdgeClassifier(nn.Module):
    def __init__(
        self,
        node_indim,
        edge_indim,
        L=4,
        node_latentdim=8,
        edge_latentdim=12,
        r_hidden_size=32,
        o_hidden_size=32,
    ):
        super().__init__()
        self.node_encoder = MLP(node_indim, node_latentdim, 64, L=1)
        self.edge_encoder = MLP(edge_indim, edge_latentdim, 64, L=1)
        gnn_layers = []
        for _l in range(L):
            # fixme: Wrong parameters?
            gnn_layers.append(
                InteractionNetwork(
                    node_latentdim,
                    edge_latentdim,
                    node_outdim=node_latentdim,
                    edge_outdim=edge_latentdim,
                    edge_hidden_dim=r_hidden_size,
                    node_hidden_dim=o_hidden_size,
                )
            )
        self.gnn_layers = nn.ModuleList(gnn_layers)
        self.W = MLP(edge_latentdim, 1, 32, L=2)

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:
        node_latent = self.node_encoder(x)
        edge_latent = self.edge_encoder(edge_attr)
        
        #print(node_latent.shape, edge_index.shape, edge_latent.shape)
        
        for layer in self.gnn_layers:
            node_latent, edge_latent = layer(node_latent, edge_index, edge_latent.squeeze(1))
            edge_latent = edge_latent.unsqueeze(1) #this squeezing and unsqueezing is a hotfix
            
        edge_weights = torch.sigmoid(self.W(edge_latent))
        return edge_weights

In [9]:
from torch_geometric.loader import DataLoader

use_cuda = False

dataframes = data_puller(1)

# Split into train and test sets
train_dfs, valid_dfs = train_test_split(dataframes, test_size=0.2)

# Create custom datasets and dataloaders
train_dataset = CustomDataset(train_dfs, use_cuda)
valid_dataset = CustomDataset(valid_dfs, use_cuda)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

In [10]:
node_indim = 3
edge_indim = 2

model = EdgeClassifier(node_indim, edge_indim)
if use_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
for epoch in range(100):
    total_loss = 0.0
    for data in train_loader:
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        output = model(data.x.float(), data.edge_index, data.edge_attr.float())
        loss = criterion(output, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        del data
        torch.cuda.empty_cache()

    print("Epoch %d, Loss = %f" % (epoch+1, total_loss/len(dataloader)))

/opt/conda/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([3882])) that is different to the input size (torch.Size([3531921, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
torch.cuda.empty_cache()